# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-08-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-16-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e) 

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

In [8]:
# Suggestion from https://stackoverflow.com/questions/41247345/python-read-cassandra-data-into-pandas

def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Table Creation and Queries



## Query 1

### Support queries such as give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Based on the query, the requirements are
+ Partition the data by session id and item in session to support filtering by session id and item in session
+ Include the columns artist, song title, and song length

In [9]:
query = "DROP TABLE IF EXISTS song_play_events"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [10]:
# Support queries such as give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS song_play_events "
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length float, PRIMARY KEY ((session_id, item_in_session)))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [11]:
# Process data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_play_events (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
# SELECT statement to verify the data was entered into the table
session.row_factory = pandas_factory
session.default_fetch_size = None

query = "SELECT artist_name, song_title, song_length from song_play_events WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = rows._current_rows
print(df)

  artist_name                       song_title  song_length
0   Faithless  Music Matters (Mark Knight Dub)   495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## Query 2

### Support queries such as give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Based on the query, the requirements are
+ Partition the data by user id and session id to support filtering by user id and session id
+ Cluster the data around item in session id to support ordering
+ Include the columns artist, song title, user first name, and user last name

In [13]:
query = "DROP TABLE IF EXISTS song_play_user_events"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [14]:
# The primary key has two parts: partition key(required) and clustering key(optional).

# Using partition key you specify how Cassandra distributes data among nodes
# Using clustering key you specify how Cassandra stores data in each node
# Specifying primary key in creating a table in the form PRIMARY KEY (f1, f2, f3, f4)
# by default, the first element (here f1) is the partition key and any field after that (here f2, f3, f4) are considered clustering key.
#
#
# Compound key:
# A multi-column primary key is called a Compound Key : Primary key (partition key, Clustering Column);

# Composite key:
# A partition key based on multiple columns : Primary key ((partition key1, partition key2), Clustering Column);
# Source: https://stackoverflow.com/questions/48206948/cassandra-no-viable-alternative-at-input    
    
# Support queries such as give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS song_play_user_events "
query = query + "(user_id int, \
                    session_id int, \
                    item_in_session int, \
                    artist_name text, \
                    song_title text, \
                    first_name text, \
                    last_name text, \
                    PRIMARY KEY ((user_id, session_id), item_in_session)) WITH CLUSTERING ORDER BY (item_in_session ASC)"
try:
    session.execute(query)
except Exception as e:
    print(e)                       

In [15]:
# Process data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_play_user_events (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [16]:
# SELECT statement to verify the data was entered into the table
session.row_factory = pandas_factory
session.default_fetch_size = None

query = "SELECT artist_name, song_title, first_name, last_name from song_play_user_events WHERE user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = rows._current_rows
print(df)

         artist_name                                         song_title  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  first_name last_name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz  


## Query 3

### Support queries such as give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Based on the query, the requirements are
+ Partition the data by song to support filtering by song
+ Song is not unique, so include user id, but not as part of the partitioning.
+ Include the columns user first name and user last name

In [17]:
query = "DROP TABLE IF EXISTS user_events"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [18]:
# Support queries such as give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS user_events "
query = query + "(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY ((song_title), user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)                           

In [19]:
# Process data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_events (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [20]:
# SELECT statement to verify the data was entered into the table
session.row_factory = pandas_factory
session.default_fetch_size = None

query = "SELECT first_name, last_name from user_events WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = rows._current_rows
print(df)

   first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


### Drop the tables before closing out the sessions

In [21]:
# Drop the table before closing out the sessions
query = "DROP TABLE IF EXISTS song_play_events"
try:
    session.execute(query)
except Exception as e:
    print(e) 

query = "DROP TABLE IF EXISTS song_play_user_events"
try:
    session.execute(query)
except Exception as e:
    print(e)    
    
query = "DROP TABLE IF EXISTS user_events"
try:
    session.execute(query)
except Exception as e:
    print(e)   

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()